<a href="https://colab.research.google.com/github/SynBoom/BERT-FOR-NER/blob/main/BERT_for_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi # kiểm tra GPU

Mon Aug  1 08:59:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:

!pip install simpletransformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 249 kB 7.0 MB/s 
     |████████████████████████████████| 4.7 MB 57.7 MB/s 
     |████████████████████████████████| 9.1 MB 39.0 MB/s 
     |████████████████████████████████| 6.6 MB 64.8 MB/s 
     |████████████████████████████████| 365 kB 24.8 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 1.2 MB 51.0 MB/s 
     |████████████████████████████████| 1.8 MB 47.3 MB/s 
     |████████████████████████████████| 101 kB 13.4 MB/s 
     |████████████████████████████████| 596 kB 48.5 MB/s 
     |████████████████████████████████| 156 kB 21.2 MB/s 
     |████████████████████████████████| 181 kB 53.2 MB/s 
     |████████████████████████████████| 63 kB 1.1 MB/s 
     |████████████████████████████████| 141 kB 71.5 MB/s 
     |████████████████████████████████| 212 kB 77.3 MB/s 
     |██████████████████████████████

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving ner_dataset.csv to ner_dataset.csv


In [ ]:

import pandas as pd
import io
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from simpletransformers.ner import NERModel,NERArgs
data = pd.read_csv(io.StringIO(uploaded['ner_dataset.csv'].decode('latin1')))

In [ ]:
data.tail(10)

,Sentence #,Word,POS,Tag
1048565,Sentence: 47958,impact,NN,O
1048566,Sentence: 47958,.,.,O
1048567,Sentence: 47959,Indian,JJ,B-gpe
1048568,Sentence: 47959,forces,NNS,O
1048569,Sentence: 47959,said,VBD,O
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O
1048574,Sentence: 47959,attack,NN,O


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   Sentence #  1048575 non-null  object
 1   Word        1048575 non-null  object
 2   POS         1048575 non-null  object
 3   Tag         1048575 non-null  object
dtypes: object(4)
memory usage: 32.0+ MB


In [ ]:
data =data.fillna(method ="ffill")

In [ ]:
data.head(30)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
5,Sentence: 1,through,IN,O
6,Sentence: 1,London,NNP,B-geo
7,Sentence: 1,to,TO,O
8,Sentence: 1,protest,VB,O
9,Sentence: 1,the,DT,O


In [ ]:
data["Sentence #"] = LabelEncoder().fit_transform(data["Sentence #"] )

In [ ]:
data.head(30)

,Sentence #,Word,POS,Tag
0,0,Thousands,NNS,O
1,0,of,IN,O
2,0,demonstrators,NNS,O
3,0,have,VBP,O
4,0,marched,VBN,O
5,0,through,IN,O
6,0,London,NNP,B-geo
7,0,to,TO,O
8,0,protest,VB,O
9,0,the,DT,O


In [ ]:
data.rename(columns={"Sentence #":"sentence_id","Word":"words","Tag":"labels"}, inplace =True)

In [ ]:
data["labels"] = data["labels"].str.upper()

In [ ]:
X= data[["sentence_id","words"]]
Y =data["labels"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2)

In [ ]:
#building up train data and test data
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})

In [ ]:
train_data

,sentence_id,words,labels
693109,24088,The,O
1014511,40438,with,O
175508,45766,of,O
122172,43037,more,O
17401,45571,Special,O
...,...,...,...
405103,9459,Pass,I-ORG
616743,20214,al-Qaida,B-ORG
357194,7049,rooted,O
333395,5845,'s,O


Model Training

In [ ]:
from simpletransformers.ner import NERModel,NERArgs

In [ ]:

label = data["labels"].unique().tolist()
label

['O',
 'B-GEO',
 'B-GPE',
 'B-PER',
 'I-GEO',
 'B-ORG',
 'I-ORG',
 'B-TIM',
 'B-ART',
 'I-ART',
 'I-PER',
 'I-GPE',
 'I-TIM',
 'B-NAT',
 'B-EVE',
 'I-EVE',
 'I-NAT']

In [ ]:
args = NERArgs()
args.num_train_epochs = 1
args.learning_rate = 2e-5
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32

In [ ]:
model = NERModel('bert', 'bert-base-cased',labels=label,args =args, use_cuda = True)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [ ]:
model.train_model(train_data,eval_data = test_data,acc=accuracy_score)

  0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1499 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


(1499, 0.2382999667233034)

In [ ]:
result, model_outputs, preds_list = model.eval_model(test_data)

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1462 [00:00<?, ?it/s]

In [ ]:
result

{'eval_loss': 0.17720016481153533,
 'f1_score': 0.7905648789070929,
 'precision': 0.8196681811923447,
 'recall': 0.7634574075855215}

In [ ]:
{'eval_loss': 0.17093349652466913,
 'precision': 0.8274360746371804,
 'recall': 0.7640470948597684,
 'f1_score': 0.7944791891309037}

{'eval_loss': 0.17093349652466913,
 'f1_score': 0.7944791891309037,
 'precision': 0.8274360746371804,
 'recall': 0.7640470948597684}

**Dự đoán với Tiếng Anh**

In [ ]:
text = "At a time when Elon Musk’s acquisition of Twitter (NYSE:TWTR) seems to be falling apart, Tesla (NASDAQ:TSLA) disclosed another subpoena sent by the SEC over Musk's tweets. Received on June 13, it is related to the 2018 tweets he made about Tesla potentially going private. Musk’s jousting with the Securities and Exchange Commission (SEC) is to be expected given that the agency is in charge of protecting investors from manipulation. Musk heads Tesla as a publicly traded company, but he also has over 100 million Twitter followers, nearly the combined populations of France and Poland. From these two starting conditions, there is a high clash potential. As a powerful communication tool, using Twitter to one’s advantage seems inevitable. At least, the SEC concluded this in the first settlement stipulation. SEC’s Problem With Musk Tweets Explained There are two kinds of impacts a billionaire like Musk can make on social media with his massive following. There is the public’s reaction to his tweets. This happened when Musk cancelled Bitcoin payments for Tesla products. Because he additionally framed it within eco-sustainability, casting doubt on Bitcoin’s fundamentals, BTC plummeted by 25% in May 2021. The second impact is more direct, with legal consequences. In September 2018, Musk settled fraud charges with the SEC for tweeting that he could take Tesla private at $420 per share and that he already secured funding to make it happen."

In [ ]:
prediction, model_output = model.predict([text])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
prediction

[[{'At': 'O'},
  {'a': 'O'},
  {'time': 'O'},
  {'when': 'O'},
  {'Elon': 'B-PER'},
  {'Musk’s': 'I-PER'},
  {'acquisition': 'O'},
  {'of': 'O'},
  {'Twitter': 'B-ORG'},
  {'(NYSE:TWTR)': 'O'},
  {'seems': 'O'},
  {'to': 'O'},
  {'be': 'O'},
  {'falling': 'O'},
  {'apart,': 'O'},
  {'Tesla': 'B-PER'},
  {'(NASDAQ:TSLA)': 'O'},
  {'disclosed': 'O'},
  {'another': 'O'},
  {'subpoena': 'O'},
  {'sent': 'O'},
  {'by': 'O'},
  {'the': 'O'},
  {'SEC': 'B-ORG'},
  {'over': 'O'},
  {"Musk's": 'I-PER'},
  {'tweets.': 'O'},
  {'Received': 'O'},
  {'on': 'O'},
  {'June': 'B-TIM'},
  {'13,': 'I-TIM'},
  {'it': 'O'},
  {'is': 'O'},
  {'related': 'O'},
  {'to': 'O'},
  {'the': 'O'},
  {'2018': 'B-TIM'},
  {'tweets': 'O'},
  {'he': 'O'},
  {'made': 'O'},
  {'about': 'O'},
  {'Tesla': 'B-PER'},
  {'potentially': 'O'},
  {'going': 'O'},
  {'private.': 'O'},
  {'Musk’s': 'B-PER'},
  {'jousting': 'O'},
  {'with': 'O'},
  {'the': 'O'},
  {'Securities': 'I-ORG'},
  {'and': 'I-ORG'},
  {'Exchange': 'I-ORG'}

In [ ]:
# Xử lý cho 1 tập các Keys in dict
import re
z = prediction[0]
#key

keys = []
for dicts in z:
  key = dicts.keys()
  keys.append(key)

# list Key
list_str = []
for i in range(len(keys)):
  list_str.append(str(keys[i]))

idx_clear = [id for id in range(0,12)]
idx_clear.append(-1) # Phần tử cuối
idx_clear.append(-2) # Phần tử thứ 2 từ cuối

my_str_key_clear = []
for j in range(len(list_str)):
  strs = list_str[j]
  my_str = ''
  strs_adj = []
  for i in range(len(strs)):
    if i not in idx_clear:
      strs_adj.append(strs[i]) 

  my_str = ''.join(strs_adj) # join to str

  new_string = re.sub(r"[^a-zA-Z0-9]","",my_str) # lọc ký tự đặc biệt

  my_str_key_clear.append(new_string)
my_str_key_clear

In [ ]:
# my_keys = []
# z = prediction[0] 
# for dicts in z: # các giá trị trong list z là các dicts
#   for mykey in dicts.keys():
#     my_keys.append(mykey)
# my_keys

In [ ]:
my_values = []
my_items = []
z = prediction[0] 
for dicts in z: # các giá trị trong list z là các dicts
  for myvalue in dicts.values():
    if myvalue !="O":
      my_items.append(dicts.items())

#>> ['Lemon', 'Orage', 'Grapes']
my_items

[dict_items([('Elon', 'B-PER')]),
 dict_items([('Musk’s', 'I-PER')]),
 dict_items([('Twitter', 'B-ORG')]),
 dict_items([('Tesla', 'B-PER')]),
 dict_items([('SEC', 'B-ORG')]),
 dict_items([("Musk's", 'I-PER')]),
 dict_items([('June', 'B-TIM')]),
 dict_items([('13,', 'I-TIM')]),
 dict_items([('2018', 'B-TIM')]),
 dict_items([('Tesla', 'B-PER')]),
 dict_items([('Musk’s', 'B-PER')]),
 dict_items([('Securities', 'I-ORG')]),
 dict_items([('and', 'I-ORG')]),
 dict_items([('Exchange', 'I-ORG')]),
 dict_items([('Commission', 'I-ORG')]),
 dict_items([('Musk', 'I-PER')]),
 dict_items([('Tesla', 'B-PER')])]

**Kiểm thử BERT với ngôn ngữ tiếng việt.**

In [ ]:
text = "Xin chào mọi người! Mình là Thành và mình đến từ Hà Nam. Hôm nay mình làm đồ án."

In [ ]:
prediction, model_output = model.predict([text])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
prediction

[[{'Xin': 'O'},
  {'chào': 'O'},
  {'mọi': 'O'},
  {'người!': 'O'},
  {'Mình': 'O'},
  {'là': 'O'},
  {'Thành': 'I-ORG'},
  {'và': 'O'},
  {'mình': 'O'},
  {'đến': 'O'},
  {'từ': 'O'},
  {'Hà': 'B-PER'},
  {'Nam.': 'I-PER'},
  {'Hôm': 'O'},
  {'nay': 'O'},
  {'mình': 'O'},
  {'làm': 'O'},
  {'đồ': 'O'},
  {'án.': 'O'}]]